<a href="https://colab.research.google.com/github/josepmatias/ONE/blob/main/Migra%C3%A7%C3%B5es/Migracoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
#import wget
import openpyxl
import zipfile

In [2]:
pip install pyxlsb

In [3]:
#actualizar nome dos ficheiros: report migrações mês

mig_up_nome="Report_Migrações_mês.xlsb"
rate_plan_nome="Rate plans.xlsb"
sfids_nome="sfids.xlsb"
kc_nome="14413-8977127-GUIA_Posições_Comerciais_Identidades2024-06-03-07-00-21.xlsb"
wise_nome="wise_cruzar_mai24.xlsb"
contrato_rate_plan_nome="QUERIE_CONTRATO_ID_ATIVO_RATE_PLAN.xlsb" #actualização colunas "tarifário actual" e "tarifário final do mês" (ex: 31/5)

In [4]:
#carregar report migrações e rate plans
from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()
uploaded3=files.upload()
uploaded4=files.upload()

Saving Report_Migrações_mês.xlsb to Report_Migrações_mês.xlsb


Saving Rate plans.xlsb to Rate plans.xlsb


Saving 14413-8977127-GUIA_Posições_Comerciais_Identidades2024-06-03-07-00-21.xlsb to 14413-8977127-GUIA_Posições_Comerciais_Identidades2024-06-03-07-00-21.xlsb


Saving sfids.xlsb to sfids.xlsb


In [5]:
migracoes=pd.read_excel(mig_up_nome, sheet_name=0)

In [6]:
migracoes2=migracoes.copy()

In [7]:
#formatar o cabeçalho, remover colunas desnecessárias e definir o nome das colunas do relatório migrações

migracoes2=migracoes2.iloc[:, 2:]

migracoes2.columns=migracoes2.iloc[0, :]

migracoes2=migracoes2.iloc[1:, :]

migracoes2.head(2)

,Year Month,Date,Contract Id,Subscriber Account Number,Contract Msisdn,Sales Force ID,Initial Rate Plan Id,Initial Rate Plan,Initial Rate Plan Segment,Initial Rate Plan Class Lvl 5,...,Final Rate Plan Id,Final Rate Plan,Final Rate Plan Segment,Final Rate Plan Class Lvl 5,Rate Plan Hierarchy 2.Rate Plan Hier 2 - Lvl 1,Initial Customer Universe Status,Create User,Change User,Change Date,Nº of Events
1,202405,45413,23450,1.10023437,351934250456,G.O04.002,2669,Pop 8 reduzidas,Rede4,PP C/ RO,...,2670,Pop 8,Rede4,PP S/ RO,Pré Pago,IN,NaN,NaN,NaN,1
2,202405,45413,38168,1.10038156,351933280396,O.000.751.137,3670,Mais Exclusivo 1GB,Mass Market Teens,Mensalidade,...,13866,Pós_NOS Mobile 20GB_DF_004,Mass Market,Pós-pago,Regular,IN,NaN,NaN,NaN,1


In [8]:
#remover #N/A
migracoes3=migracoes2.copy()

migracoes3=migracoes3.dropna(subset="Year Month")

print(f"there are {migracoes3.isna().sum()} missing values")

print(migracoes3.shape)

there are 0
Year Month                                                     0
Date                                                           0
Contract Id                                                    0
Subscriber Account Number                                      0
Contract Msisdn                                             5291
Sales Force ID                                                 0
Initial Rate Plan Id                                           0
Initial Rate Plan                                              0
Initial Rate Plan Segment                                   4560
Initial Rate Plan Class Lvl 5                              17362
Initial Rate Plan Hierarchy 2.Rate Plan Hier 2 - Lvl 1      4560
Final Rate Plan Id                                             0
Final Rate Plan                                                0
Final Rate Plan Segment                                        0
Final Rate Plan Class Lvl 5                                12773
Rate Plan Hie

In [9]:
#reordenar colunas "Contract Id" (ascending) e Date (descending)

for i in ["Contract Id", "Date"]:
  migracoes3[i]=migracoes3[i].astype("int")

print(migracoes3.dtypes)

migracoes3=migracoes3.sort_values(by=["Contract Id", "Date"], ascending=[True, False])

0
Year Month                                                object
Date                                                       int64
Contract Id                                                int64
Subscriber Account Number                                 object
Contract Msisdn                                           object
Sales Force ID                                            object
Initial Rate Plan Id                                      object
Initial Rate Plan                                         object
Initial Rate Plan Segment                                 object
Initial Rate Plan Class Lvl 5                             object
Initial Rate Plan Hierarchy 2.Rate Plan Hier 2 - Lvl 1    object
Final Rate Plan Id                                        object
Final Rate Plan                                           object
Final Rate Plan Segment                                   object
Final Rate Plan Class Lvl 5                               object
Rate Plan Hierarchy 2.R

In [10]:
print(migracoes3.shape)

migracoes3.head(3)

(533570, 21)


,Year Month,Date,Contract Id,Subscriber Account Number,Contract Msisdn,Sales Force ID,Initial Rate Plan Id,Initial Rate Plan,Initial Rate Plan Segment,Initial Rate Plan Class Lvl 5,...,Final Rate Plan Id,Final Rate Plan,Final Rate Plan Segment,Final Rate Plan Class Lvl 5,Rate Plan Hierarchy 2.Rate Plan Hier 2 - Lvl 1,Initial Customer Universe Status,Create User,Change User,Change Date,Nº of Events
97203,202405,45420,2059,1.10002062,351933201864,O.000.763.163,14239,Pós_NOS Mobile 30GB_Ret_1L_DF_004,Mass Market,Pós-pago,...,14240,Pós_NOS Mobile Sem Limites Max_Ret_1L_DF_004,Mass Market,Pós-pago,Regular,IN,NaN,NaN,NaN,1
372592,202405,45440,3802,1.10003804,351936200584,D.S05.RI1.003,12595,Pós_NOS Mobile 50GB,Mass Market,Pós-pago,...,13867,Pós_NOS Mobile 50GB_DF_004,Mass Market,Pós-pago,Regular,IN,NaN,NaN,NaN,1
158215,202405,45425,7609,1.10007610,351934201225,G.R32.CAC.0SU,58437,Pós_NOS Mobile 5GB,Mass Market,Pós-pago,...,14239,Pós_NOS Mobile 30GB_Ret_1L_DF_004,Mass Market,Pós-pago,Regular,IN,NaN,NaN,NaN,1


In [11]:
#importar ficheiro rate_plans, definir nome das colunas, eliminar 1ª linha
rate_plans=pd.read_excel(rate_plan_nome, sheet_name="export")

rate_plans.columns=rate_plans.iloc[0,:]

rate_plans=rate_plans.iloc[1:, ]

print(rate_plans.shape)

(58327, 20)


In [12]:
#formatar o nome das colunas do ficheiro rate_plans de forma a não terem espaços em branco

rate_plans.columns=[i.strip() for i in rate_plans.columns.tolist()]

In [13]:
#substituir cada vazio na dataframe "rate_plans" por zero (colunas "RATE PLAN CLASS LVL 8 (*)" e "RATE PLAN CLASS LVL 7 (*)")

for i in ["RATE PLAN CLASS LVL 8 (*)", "RATE PLAN CLASS LVL 7 (*)"]:
  rate_plans[i]=rate_plans[i].fillna(0)


In [14]:
#criar coluna "tarifário inicial" - 1º elemento é a coluna no report migrações, 2º elemento é a coluna lookup no ficheiro rate plan e 3º elemento é a coluna que queremos no rate plans

dici={"Tarifario Inicial":["Initial Rate Plan Id", "RATE PLAN ID", "RATE PLAN CLASS LVL 8 (*)"], "Tarifario Final": ["Final Rate Plan Id", "RATE PLAN ID", "RATE PLAN CLASS LVL 8 (*)"], \
"Rede Inicial": ["Initial Rate Plan Id", "RATE PLAN ID", "RATE PLAN CLASS LVL 7 (*)"], "Rede Final": ["Final Rate Plan Id", "RATE PLAN ID", "RATE PLAN CLASS LVL 7 (*)"]}


for item1, item2 in dici.items():
  #item1 é a nova coluna a ser criada no report de migrações
  mig, rate1, rate2=item2
  #mig é a coluna no report migrações, rate1 é a coluna lookup no report rate plans, rate2 é a coluna que queremos ir buscar ao rate plan
  #fazer loop pelos valores mig

  for val in set(migracoes3[mig].tolist()):
    #identificar o índice no report de migrações
    indice=migracoes3[migracoes3[mig]==val].index
    variavel=rate_plans.loc[rate_plans[rate1]==val, rate2].unique().tolist()[0]
    migracoes3.loc[indice, item1]=variavel



In [15]:
set(migracoes3.loc[migracoes3["Tarifario Inicial"].isna()==True, "Initial Rate Plan Id"])

set()

In [16]:

#criar coluna "Contagem"

migracoes4=migracoes3.groupby("Contract Id").size().reset_index(name='Contagem').merge(migracoes3, how="right", on="Contract Id")



In [17]:
#criar copia das bds com contagem maior do que 1 - criar coluna índice

temporario=migracoes4[migracoes4["Contagem"]>1]
temporario2=pd.DataFrame(columns=["Contract Id", "Indice"])

for i in temporario["Contract Id"].unique():
  contar=temporario.loc[temporario["Contract Id"]==i, "Contagem"].values[0]
  for x, y in zip([i]*contar, range(1, contar+1)):
    dici={"Contract Id":[x], "Indice":[y]}
    temporario2=pd.concat([temporario2, pd.DataFrame(dici)], axis=0, ignore_index=True)


In [18]:
#adicionar coluna índice à dataframe "migracoes4"

migracoes4.loc[migracoes4["Contagem"]>1, "Indice"]=temporario2["Indice"].values

migracoes4.loc[migracoes4["Contagem"]==1, "Indice"]=1



In [19]:
#ler dataframe com os sfids

sfids=pd.read_excel(sfids_nome, sheet_name=0)

In [20]:
#detectar quais os sfids do report de migrações que não estão na folha "sfid"

migracoes4["Sales Force ID"].unique()

sfids_falta=pd.DataFrame(migracoes4.loc[~migracoes4["Sales Force ID"].isin(sfids["Sales Force ID"]), "Sales Force ID"].unique().tolist(), columns=["sfids em falta"])

print(sfids_falta.shape)


sfids_falta.head()

(343, 1)


,sfids em falta
0,O.000.774.100
1,O.000.063.161
2,B.C01.UP2.0DE
3,X.V03.IN1.07L
4,O.000.755.065


In [21]:
#ler relatório kc

kc_report=pd.read_excel(kc_nome, sheet_name=0)

In [22]:
#sfids_db tem os registos dos sfids em falta que se encontram no relatório kc 14413

sfids_db=kc_report.loc[kc_report["SFID Identidade (COC)"].isin(sfids_falta["sfids em falta"]), ["SFID Identidade (COC)", "Sub Canal (COC)", "Parceiro (COC)"]].drop_duplicates()


sfids_db.head(2)



,SFID Identidade (COC),Sub Canal (COC),Parceiro (COC)
233,X.V03.IN1.07L,Inbound,"RANDSTAD II - PRESTAÇÃO DE SERVIÇOS, LDA"
343,X.V03.IW1.06Z,Web,"RANDSTAD II - PRESTAÇÃO DE SERVIÇOS, LDA"


In [23]:
#dataframe sfids_falta é um merge do relatório kc com a dataframe "sfids_falta"
sfids_falta=sfids_falta.merge(right=kc_report, how="left", left_on="sfids em falta", right_on="SFID Identidade (COC)")

#sfids_falta2 é a dataframe sfids_falta sem duplicados
sfids_falta2=sfids_falta[["sfids em falta", "Sub Canal (COC)", "Parceiro (COC)"]].drop_duplicates()

print(sfids_falta2.shape)
print("estas são as linhas duplicadas na db sfids_falta2")
sfids_falta2[sfids_falta2["sfids em falta"].duplicated(False)==True]

(350, 3)
estas são as linhas duplicadas na db sfids_falta2


,sfids em falta,Sub Canal (COC),Parceiro (COC)
163,X.V03.IL2.009,Programas Internos,"RANDSTAD II - PRESTAÇÃO DE SERVIÇOS, LDA"
164,X.V03.IL2.009,Inbound,"RANDSTAD II - PRESTAÇÃO DE SERVIÇOS, LDA"
412,G.BEL.026,Alto Impacto,"RANDSTAD II - PRESTAÇÃO DE SERVIÇOS, LDA"
413,G.BEL.026,Field Sales,"RANDSTAD II - PRESTAÇÃO DE SERVIÇOS, LDA"
487,S.UNI.SAC.03W,Inbound Alarmes,NaN
488,S.UNI.SAC.03W,Outbound Fidelização,NaN
491,S.UNI.SAC.03W,Inbound Serviço a Cliente,NaN
510,D.E05.857,Agentes Norte,EASY CONNECT UNIPESSOAL LDA
512,D.E05.857,Agentes Porto,EASY CONNECT UNIPESSOAL LDA
528,P.N03.OPR.007,Agentes Nacionais,"NOPTIS - SISTEMAS DE COMUNICAÇÕES, LDA"


In [24]:
dici={"Fora Modelo Comissional": \
 {"Sub Canal (COC)": ["Inbound Serviço a Cliente", "Não Técnico", "MAIS", "Lojas Próprias", "Franchising", "Retenção", "Outbound Fidelização"]},\
 "TP":{"Sub Canal (COC)": "Outbound", "Parceiro (COC)":"Teleperformance"}, "TP2":{"Sub Canal (COC)":"Outbound Teleperformance"}, \
 "Retenção": {"Sub Canal (COC)": "Fidelização", "sfids em falta": [".RI", ".RO"]}, \
 "ONE": {"Sub Canal (COC)": "Outbound", "Parceiro (COC)": "Teleperformance", "sfids em falta": ".UP"}, \
 "INB&WEB": {"Sub Canal (COC)": "Web", "sfids em falta": [".IW", "L.F"]}, \
 "INB&WEB2":{"Sub Canal (COC)": "Inbound", "sfids em falta": ".IN"}}



for i, j in dici.items():
  if i=="Fora Modelo Comissional":
    for x, y in dici[i].items():
      for count in y:
        if sfids_falta2[sfids_falta2[x]==count].count().values[0]>0:
          indice=sfids_falta2[sfids_falta2[x]==count].index
          sfids_falta2.loc[indice, "Modelo Comissional"]=i
  elif i=="TP":
    col1, col2=dici[i].keys()
    val1, val2=dici[i].values()
    indice=sfids_falta2[(sfids_falta2[col1]==val1) & (sfids_falta2[col2].str.contains(val2, case=False))].index
    sfids_falta2.loc[indice, "Modelo Comissional"]=i

  elif i=="TP2":
    col1=list(dici[i].keys())[0]
    val1=list(dici[i].values())[0]
    if sfids_falta2[sfids_falta2[col1]==val1].count().values[0]>0:
      indice=sfids_falta2[sfids_falta2[col1]==val1].index
      sfids_falta2.loc[indice, "Modelo Comissional"]="TP"

  elif i=="Retenção":
    col1, col2=dici[i].keys()
    val1, val2=dici[i].values()

    for k in val2:
      indice=sfids_falta2[(sfids_falta2[col1]==val1) & (sfids_falta2[col2].str.find(k)!=-1)].index
      sfids_falta2.loc[indice, "Modelo Comissional"]="Retenção"

  elif i=="ONE":
    col1, col2, col3=dici[i].keys()
    val1, val2, val3=dici[i].values()

    indice=sfids_falta2[(sfids_falta2[col1]==val1) & (sfids_falta2[col2]!=val2) & (sfids_falta2[col3].str.find(val3)!=-1)].index
    sfids_falta2.loc[indice, "Modelo Comissional"]="ONE"

  elif i=="INB&WEB":
    col1, col2=dici[i].keys()
    val1, lista1=dici[i].values()
    #val2, val3=lista1
    if sfids_falta2[sfids_falta2[col1]==val1].count().values[0]>0:
      for count2 in range(len(lista1)):
        if count2==0:
          indice=sfids_falta2[(sfids_falta2[col1]==val1) & (sfids_falta2[col2].str.find(lista1[count2])!=-1)].index
          sfids_falta2.loc[indice, "Modelo Comissional"]="INB&WEB"

        else:
          indice=sfids_falta2[(sfids_falta2[col1]==val1) & (sfids_falta2[col2].str.startswith(lista1[count2])==True)].index
          sfids_falta2.loc[indice, "Modelo Comissional"]="Fora Modelo Comissional"

  else:
    col1, col2=dici[i].keys()
    val1, val2=dici[i].values()

    if sfids_falta2[sfids_falta2[col1]==val1].count().values[0]>0:
      indice=sfids_falta2[(sfids_falta2[col1]==val1) & (sfids_falta2[col2].str.find(val2)!=-1)].index
      sfids_falta2.loc[indice, "Modelo Comissional"]="INB&WEB"

In [25]:
#converter aqueles sfids cujo modelo comissional não foi encontrado como "Fora Modelo Comissional"

indice_fora_modelo_comissional=sfids_falta2[sfids_falta2["Modelo Comissional"].isna()==True].index

sfids_falta2.loc[indice_fora_modelo_comissional, "Modelo Comissional"]="Fora Modelo Comissional"

#eliminar linhas com sfids duplicados

indice_duplicado=sfids_falta2[sfids_falta2["sfids em falta"].duplicated()==True].index

print(sfids_falta2.shape)

sfids_falta3=sfids_falta2.drop(index=sfids_falta2[sfids_falta2["sfids em falta"].duplicated()==True].index)

print(sfids_falta3.shape)

(350, 4)
(343, 4)


In [26]:
#contabilizar quantas linhas cada modelo comissional tem nos "sfids em falta"
sfids_falta3["Modelo Comissional"].value_counts()

Modelo Comissional
Fora Modelo Comissional    305
ONE                         14
INB&WEB                     13
Retenção                    10
TP                           1
Name: count, dtype: int64

In [27]:
#reordenar e re-nomear colunas da dataframe "sfids_falta3"
colunas_new=["sfids em falta", "Modelo Comissional", "Sub Canal (COC)", "Parceiro (COC)"]

sfids_falta3=sfids_falta3[colunas_new]

sfids_falta3.rename(columns={"sfids em falta": "Sales Force ID"}, inplace=True)


sfids_falta3.head()

,Sales Force ID,Modelo Comissional,Sub Canal (COC),Parceiro (COC)
0,O.000.774.100,Fora Modelo Comissional,Lojas Próprias,"MANPOWERGROUP SOLUTIONS, UNIPESSOAL LDA"
1,O.000.063.161,Fora Modelo Comissional,Lojas Próprias,"MANPOWERGROUP SOLUTIONS, UNIPESSOAL LDA"
3,B.C01.UP2.0DE,ONE,Outbound,"RANDSTAD II - PRESTAÇÃO DE SERVIÇOS, LDA"
5,X.V03.IN1.07L,INB&WEB,Inbound,"RANDSTAD II - PRESTAÇÃO DE SERVIÇOS, LDA"
8,O.000.755.065,Fora Modelo Comissional,Lojas Próprias,"RANDSTAD II - PRESTAÇÃO DE SERVIÇOS, LDA"


In [28]:
#juntar dataframe "sfids_falta3" à dataframe inicial dos sfids ("sfids")

sfids_completo=pd.concat([sfids, sfids_falta3], axis=0)

#remover "L.G01" com modelo comissional = ONE

#sfids_completo.drop(index=sfids_completo[(sfids_completo["Sales Force ID"]=="L.G01") & sfids_completo["Modelo Comissional"]=="ONE"].index, inplace=True)

sfids_completo["Modelo Comissional"]=sfids_completo["Modelo Comissional"].str.strip()

sfids_completo["Modelo Comissional"].value_counts()


Modelo Comissional
Fora Modelo Comissional    22093
ONE                         3176
Retenção                     786
INB&WEB                      502
TP                           153
Name: count, dtype: int64

In [29]:
#eliminar linha com sfid"L.G01" e modelo de comissionamento igual a "ONE"
print(sfids_completo.shape)
sfids_completo2=sfids_completo.drop(sfids_completo.loc[(sfids_completo["Sales Force ID"]=="L.G01") & (sfids_completo["Modelo Comissional"]=="ONE"), :].index)
print(sfids_completo2.shape)

sfids_completo2[sfids_completo2.iloc[:, 0]=="L.G01"]

(26710, 5)
(26709, 5)


,Sales Force ID,Modelo Comissional,Sub Canal (COC),Parceiro (COC),(Inf Kc14413)
487,L.G01,Fora Modelo Comissional,0x2a,0x2a,NaN


In [30]:
#identificar o modelo comissional do report migrações do mês n - coluna "modelo comissional"

def modelo_comissional (db, sfid_db, str="Modelo Comissional", str2="Sales Force ID"):

  for i in set(sfid_db[str]):
    #variável "sfids" são aqueles que estão como "fora do modelo comissional" na folha sfids
    sfids=sfid_db.loc[sfid_db[str]==i, str2].values
    mask=db[str2].isin(sfids)
    indice=db[mask].index
    print(f"{i} \n {len(indice)}")
    db.loc[indice, "Modelo Comissional"]=i

modelo_comissional(migracoes4, sfids_completo2)

Retenção 
 3944
TP 
 105
ONE 
 4875
Fora Modelo Comissional 
 524546
INB&WEB 
 100


In [31]:
#fazer upload ficheiro query_contrato_id_ativo_rate_plan e ficheiro wise para o mês n

from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()

Saving QUERIE_CONTRATO_ID_ATIVO_RATE_PLAN.xlsb to QUERIE_CONTRATO_ID_ATIVO_RATE_PLAN.xlsb


Saving wise_cruzar_mai24.xlsb to wise_cruzar_mai24.xlsb


In [32]:
#ler ficheiro querie_contrato_id_ativo_rate_plan
query_contrato=pd.read_excel(contrato_rate_plan_nome, sheet_name=0)

In [33]:
#formatar dataframe "query_contrato": definir nomes das colunas e eliminar índices a mais

query_contrato2=query_contrato.iloc[2:, 1:]
query_contrato2.columns=query_contrato2.iloc[0, :]

query_contrato2.index=range(query_contrato2.iloc[:, 0].count())

query_contrato2.drop(labels=0, axis=0, inplace=True)

In [34]:
#ver as primeiras linhas da dataframe query_contrato2
query_contrato2.head()

2,Contract Id,Rate Plan,Rate Plan Id,EOP DW With Offers,Rate Plan (F),Rate Plan Id (F)
1,2059,Pós_NOS Mobile Sem Limites Max_Ret_1L_DF_004,14240,1,Pós_NOS Mobile 30GB_Ret_1L_DF_004,14239
2,3802,Pós_NOS Mobile 50GB_DF_004,13867,1,Pós_NOS Mobile 50GB,12595
3,7609,Pós_NOS Mobile 30GB_Ret_1L_DF_004,14239,1,Pós_NOS Mobile 5GB,58437
4,9729,Pós_NOS Mobile 1GB_004,14231,1,Livres Base,3345
5,11632,Pós_NOS Mobile 10GB_DF_004,13865,1,Mais Exclusivo,3351


In [35]:
#criar colunas "tarifário actual" e "tarifário final do mês" na dataframe "migracoes4"

migracoes5=migracoes4.merge(query_contrato2[["Contract Id", "Rate Plan", "Rate Plan (F)"]], how="left", on="Contract Id")

migracoes5.columns=migracoes5.columns.tolist()[:-2] + ["tarifario actual", "tarifario fim mes"]

In [36]:
#criar dicionário com rate plan e o tipo correspondente

rate_plans_dup=rate_plans.iloc[:, [1, 9]].drop_duplicates()


dici_tarif={}

for i, j in zip(rate_plans_dup.iloc[:, 0], rate_plans_dup.iloc[:, 1]):
  dici_tarif[i]=str(j).strip()






In [37]:
#rate plans com mais do que um valor para "rate plan class lvl 8 (*)"
rate_plans_dup[rate_plans_dup.iloc[:, 0].duplicated(False)].sort_values("RATE PLAN")

,RATE PLAN,RATE PLAN CLASS LVL 8 (*)
6009,Kanguru 7,Pós-Pago 2
58156,Kanguru 7,Pós-Pago 1
2151,Kanguru Zero,Pós-Pago
2956,Kanguru Zero,Pré-Pago
1493,Livre 75,Pré-Pago
1997,Livre 75,Pós-Pago 1
13658,MPRO,0
13705,MPRO,ICT
57497,NAO USAR,0
57837,NAO USAR,Pós-Pago 2


In [38]:
#criar coluna "tipo"

migracoes5["tipo"]=migracoes5.loc[migracoes5["tarifario actual"].str.strip().isin(list(dici_tarif.keys())), "tarifario actual"].apply(lambda i: dici_tarif[i])

In [39]:
#criar coluna "validação" que compara o tarifário actual com o tarifário de final de mês
migracoes5["validacao"]=migracoes5["tarifario actual"]==migracoes5["tarifario fim mes"]

In [40]:
#ler ficheiro wise

wise=pd.read_excel(wise_nome, sheet_name=0)
wise["conta_servico"]=wise["conta_servico"].astype("str")
wise.head()

,id,id_navenda,dsc_operacao,callid,data,data_insercao_fo,Novo_ja_cliente,conta_cliente,conta_servico,servico_atual,...,ID Motivo FE Papel,Motivo FE Papel,CAV_venda,data_agendamento,id_tipo_telefone_fixo,id_tipo_router,tipo_registo,nova_encomenda,pvp_marcacao,id_pessoa
0,8073781,77234103,ONE,EquipaInterna_77234103_1a01a8c00000119466435de...,45426,45426.584757,NaN,NaN,17563,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,BOC (Default),NaN,NaN,132338920
1,8023024,76871223,ONE,EquipaInterna_76871223_1a01a8c000000bbc660c2c1...,45384,45384.716204,NaN,NaN,26065,NaN,...,6.0,Contabilidade: Necessita que seja em papel par...,NaN,NaN,NaN,NaN,BOC (Default),NaN,NaN,116503327
2,8020030,76748600,ONE,EquipaInterna_76748600_1a01a8c0000013dc660a84b...,45383,45383.468970,NaN,NaN,26312,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,BOC (Default),NaN,NaN,211003808
3,8034919,76991758,ONE,EquipaInterna_76991758_1a01a8c0000022ec6617f5e...,45393,45393.656933,NaN,NaN,27456,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,BOC (Default),NaN,NaN,203274652
4,8022633,76870716,ONE,EquipaInterna_76870716_1a01a8c000000bbc660c0fa...,45384,45384.632917,NaN,NaN,46172,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,BOC (Default),NaN,NaN,129048950


In [91]:
#criar colunas wise: cav_operador, data, campanha_nome, estado_actual e dsc_operacao
col_contrato="conta_servico"
cols_wise=["CAV_operador", "data", "campanha_nome", "estado_atual", "dsc_operacao"]

def conv_wise(db1, db2, col1="Contract Id", col2=col_contrato, col3=cols_wise):
  for i in col3:
    db1[col1]=db1[col1].astype("str")
    db2[col2]=db2[col2].astype("str")
    db1[i]=db1[[col1]].merge(db2[[col2, i]], how="left", left_on=col1, right_on=col2).drop([col1, col2], axis=1)
    if i=="data":
      #data formatada como texto
      db1[i]=pd.to_datetime(db1[i], unit="d", origin="1899-12-30")
      db1[i]=db1.loc[db1[i].isna()==False, i].apply(lambda i: i.strftime("%d-%m-%Y"))
    elif i=="CAV_operador":
      db1[i]=db1.loc[db1[i].isna()==False, i].astype("int").astype("str")
    else:
      db1[i]=db1[i].astype("str")

  return db1

conv_wise(migracoes5, wise)

,Contract Id,Contagem,Year Month,Date,Subscriber Account Number,Contract Msisdn,Sales Force ID,Initial Rate Plan Id,Initial Rate Plan,Initial Rate Plan Segment,...,Modelo Comissional,tarifario actual,tarifario fim mes,tipo,validacao,CAV_operador,data,campanha_nome,estado_atual,dsc_operacao
0,2059,1,202405,45420,1.10002062,351933201864,O.000.763.163,14239,Pós_NOS Mobile 30GB_Ret_1L_DF_004,Mass Market,...,Fora Modelo Comissional,Pós_NOS Mobile Sem Limites Max_Ret_1L_DF_004,Pós_NOS Mobile 30GB_Ret_1L_DF_004,Pós-Pago,False,NaN,NaN,nan,nan,nan
1,3802,1,202405,45440,1.10003804,351936200584,D.S05.RI1.003,12595,Pós_NOS Mobile 50GB,Mass Market,...,Retenção,Pós_NOS Mobile 50GB_DF_004,Pós_NOS Mobile 50GB,Pós-Pago,False,NaN,NaN,nan,nan,nan
2,7609,1,202405,45425,1.10007610,351934201225,G.R32.CAC.0SU,58437,Pós_NOS Mobile 5GB,Mass Market,...,Fora Modelo Comissional,Pós_NOS Mobile 30GB_Ret_1L_DF_004,Pós_NOS Mobile 5GB,Pós-Pago,False,NaN,NaN,nan,nan,nan
3,9729,1,202405,45414,1.10009732,351938401341,O.000.763.149,3345,Livres Base,Rede4,...,Fora Modelo Comissional,Pós_NOS Mobile 1GB_004,Livres Base,Pós-Pago,False,NaN,NaN,nan,nan,nan
4,11632,1,202405,45441,1.10011636,351933204245,A.G05.WF4.058,3351,Mais Exclusivo,Mass Market Teens,...,Fora Modelo Comissional,Pós_NOS Mobile 10GB_DF_004,Mais Exclusivo,Pós-Pago,False,NaN,NaN,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533565,222530969,1,202405,45443,1.83105856,351932172624,A.W01.013,11941,Pre WTF RPOferta 7D promo 20,WTF,...,Fora Modelo Comissional,NaN,NaN,NaN,False,NaN,NaN,nan,nan,nan
533566,223049907,1,202405,45436,1.83124786,351934053174,F.005,13988,WOO_Móvel 15GB + 500cred,Mass Market,...,Fora Modelo Comissional,NaN,NaN,NaN,False,NaN,NaN,nan,nan,nan
533567,223076911,1,202405,45439,1.83151791,351936636956,F.005,14203,Pré_Like 5GB pré-ativação_PG_004,Mass Market,...,Fora Modelo Comissional,NaN,NaN,NaN,False,NaN,NaN,nan,nan,nan
533568,223079961,1,202405,45439,1.83154841,351936678830,F.005,14204,Pré_Like 5GB pré-ativação_014,Mass Market,...,Fora Modelo Comissional,NaN,NaN,NaN,False,NaN,NaN,nan,nan,nan


In [111]:
#criar coluna "Dif Datas Wise_Mig"

def data_conversor(db, col1="Date", col2="data"):
  data_migracao=pd.to_datetime(migracoes5["Date"], unit="D", origin="1899-12-30")
  data_wise=pd.to_datetime(migracoes5.loc[migracoes5["data"].isna()==False, "data"])

  db["dif_datas_wise"]=(data_wise-data_migracao).dt.days

  return db
data_conversor(migracoes5)


<ipython-input-111-7ccb73ad7a31>:7: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data_wise=pd.to_datetime(migracoes5.loc[migracoes5["data"].isna()==False, "data"])


,Contract Id,Contagem,Year Month,Date,Subscriber Account Number,Contract Msisdn,Sales Force ID,Initial Rate Plan Id,Initial Rate Plan,Initial Rate Plan Segment,...,tarifario actual,tarifario fim mes,tipo,validacao,CAV_operador,data,campanha_nome,estado_atual,dsc_operacao,dif_datas_wise
0,2059,1,202405,45420,1.10002062,351933201864,O.000.763.163,14239,Pós_NOS Mobile 30GB_Ret_1L_DF_004,Mass Market,...,Pós_NOS Mobile Sem Limites Max_Ret_1L_DF_004,Pós_NOS Mobile 30GB_Ret_1L_DF_004,Pós-Pago,False,NaN,NaN,nan,nan,nan,NaN
1,3802,1,202405,45440,1.10003804,351936200584,D.S05.RI1.003,12595,Pós_NOS Mobile 50GB,Mass Market,...,Pós_NOS Mobile 50GB_DF_004,Pós_NOS Mobile 50GB,Pós-Pago,False,NaN,NaN,nan,nan,nan,NaN
2,7609,1,202405,45425,1.10007610,351934201225,G.R32.CAC.0SU,58437,Pós_NOS Mobile 5GB,Mass Market,...,Pós_NOS Mobile 30GB_Ret_1L_DF_004,Pós_NOS Mobile 5GB,Pós-Pago,False,NaN,NaN,nan,nan,nan,NaN
3,9729,1,202405,45414,1.10009732,351938401341,O.000.763.149,3345,Livres Base,Rede4,...,Pós_NOS Mobile 1GB_004,Livres Base,Pós-Pago,False,NaN,NaN,nan,nan,nan,NaN
4,11632,1,202405,45441,1.10011636,351933204245,A.G05.WF4.058,3351,Mais Exclusivo,Mass Market Teens,...,Pós_NOS Mobile 10GB_DF_004,Mais Exclusivo,Pós-Pago,False,NaN,NaN,nan,nan,nan,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533565,222530969,1,202405,45443,1.83105856,351932172624,A.W01.013,11941,Pre WTF RPOferta 7D promo 20,WTF,...,NaN,NaN,NaN,False,NaN,NaN,nan,nan,nan,NaN
533566,223049907,1,202405,45436,1.83124786,351934053174,F.005,13988,WOO_Móvel 15GB + 500cred,Mass Market,...,NaN,NaN,NaN,False,NaN,NaN,nan,nan,nan,NaN
533567,223076911,1,202405,45439,1.83151791,351936636956,F.005,14203,Pré_Like 5GB pré-ativação_PG_004,Mass Market,...,NaN,NaN,NaN,False,NaN,NaN,nan,nan,nan,NaN
533568,223079961,1,202405,45439,1.83154841,351936678830,F.005,14204,Pré_Like 5GB pré-ativação_014,Mass Market,...,NaN,NaN,NaN,False,NaN,NaN,nan,nan,nan,NaN


In [128]:
#criar coluna "tipo migração" (ex: "pré-pago para pré-pago") - VERIFICAR

migracoes5["tipo migracao"]=migracoes5["Tarifario Inicial"].astype("str")  + " para " + migracoes5["Tarifario Final"].astype("str")


In [174]:
#análise daf das linhas com contagem >1
migracoes5.loc[migracoes5["Contagem"]>1, "analise daf"]="nao remunerado:duplicado"



In [186]:
#análise daf das linhas fora do modelo comissional

#não remunerado: fora do modelo comissional (tipo de tarifario) -> condições 1, 2, 3 e 4
condicao1=migracoes5["Modelo Comissional"]=="Fora Modelo Comissional"
condicao2=~migracoes5["Initial Rate Plan Segment"].isin(["Mass Market", "Mass Market Teens", "Rede4", "WTF"])
condicao3=~migracoes5["Final Rate Plan Segment"].isin(["Mass Market", "Mass Market Teens", "Rede4", "WTF"])
condicao4=migracoes5["Indice"]==1

#não remunerado: fora do modelo comissional (tipo de tarifario) -> condição 1, 4, 5, 6
condicao5=migracoes5["Final Rate Plan Segment"]=="Outros DUMMY"
condicao6=migracoes5["Initial Rate Plan Segment"]==""


#não remunerado: fora do modelo comissional (tipo de tarifario) -> condições 1, 4, 7, 8, 9, 10
condicao7=migracoes5["Initial Rate Plan Segment"].isin(["Mass Market", "Mass Market Teens", "Rede4", "WTF"])
condicao8=migracoes5["Final Rate Plan Segment"].isin(["Mass Market", "Mass Market Teens", "Rede4", "WTF"])
condicao9=migracoes5["Rede Inicial"]=="GO Móvel"
condicao10=migracoes5["Rede Final"]=="GO Móvel"


#migracoes5[condicao1 & condicao2 & condicao3 & condicao4]

,Contract Id,Contagem,Year Month,Date,Subscriber Account Number,Contract Msisdn,Sales Force ID,Initial Rate Plan Id,Initial Rate Plan,Initial Rate Plan Segment,...,tipo,validacao,CAV_operador,data,campanha_nome,estado_atual,dsc_operacao,dif_datas_wise,tipo migracao,analise daf
235,763259,1,202405,45441,5.69186.00.00.100001,351932205087,G.G01.929,58623,Plano Mobilidade Sem Limites Max EEE Mcentrex_23,PME,...,Pós-Pago 2,False,NaN,NaN,nan,nan,nan,NaN,Pós-Pago 2 para Pós-Pago 2,NaN
379,1295392,1,202405,45441,5.77609.00.00.100001,351936028550,P.N03.EBI,13842,Plano Móvel Pro_Mcentrex_60GB 4000 min/sms,PME,...,Pós-Pago 2,False,NaN,NaN,nan,nan,nan,NaN,Pós-Pago 2 para Pós-Pago 2,NaN
489,1641672,1,202405,45436,5.65670.00.00.100001,351936224476,G.M44.P02.969,14803,Plano Móvel Pro_Mcentrex_60GB 4000 min/sms_OA24,PME,...,Pós-Pago 2,False,NaN,NaN,nan,nan,nan,NaN,Pós-Pago 2 para Pós-Pago 2,NaN
740,2412834,1,202405,45440,5.66765.00.00.100001,351937560631,G.THN.S03.008,12613,Plano Mobilidade 120GB 20000Min/SMS EEE Mcentr...,PME,...,Pós-Pago 2,False,NaN,NaN,nan,nan,nan,NaN,Pós-Pago 2 para Pós-Pago 2,NaN
741,2412838,1,202405,45440,5.66765.00.00.100001,351937560632,G.THN.S03.008,12612,Plano Mobilidade 60GB 20000Min/SMS EEE Mcentre...,PME,...,Pós-Pago 2,False,NaN,NaN,nan,nan,nan,NaN,Pós-Pago 2 para Pós-Pago 2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533549,222491488,1,202405,45441,1.83066375,351935159073,S.W63,6161,Transitorio Convergente,GO Conv Móvel Resid,...,NaN,False,NaN,NaN,nan,nan,nan,NaN,Pós-Pago para Pós-Pago,NaN
533550,222491496,1,202405,45440,1.83066383,351935161496,O.000.757,6161,Transitorio Convergente,GO Conv Móvel Resid,...,NaN,False,NaN,NaN,nan,nan,nan,NaN,Pós-Pago para Pós-Pago,NaN
533552,222491546,1,202405,45439,1.83066433,351935163426,A.P40.TI8,6161,Transitorio Convergente,GO Conv Móvel Resid,...,NaN,False,NaN,NaN,nan,nan,nan,NaN,Pós-Pago para Pós-Pago,NaN
533553,222491795,1,202405,45442,1.83066682,351935165215,O.000.785,6161,Transitorio Convergente,GO Conv Móvel Resid,...,NaN,False,NaN,NaN,nan,nan,nan,NaN,Pós-Pago para Pós-Pago,NaN


In [183]:
print(migracoes5.columns)

migracoes5["Modelo Comissional"].unique()

Index(['Contract Id', 'Contagem', 'Year Month', 'Date',
       'Subscriber Account Number', 'Contract Msisdn', 'Sales Force ID',
       'Initial Rate Plan Id', 'Initial Rate Plan',
       'Initial Rate Plan Segment', 'Initial Rate Plan Class Lvl 5',
       'Initial Rate Plan Hierarchy 2.Rate Plan Hier 2 - Lvl 1',
       'Final Rate Plan Id', 'Final Rate Plan', 'Final Rate Plan Segment',
       'Final Rate Plan Class Lvl 5',
       'Rate Plan Hierarchy 2.Rate Plan Hier 2 - Lvl 1',
       'Initial Customer Universe Status', 'Create User', 'Change User',
       'Change Date', 'Nº of Events', 'Tarifario Inicial', 'Tarifario Final',
       'Rede Inicial', 'Rede Final', 'Indice', 'Modelo Comissional',
       'tarifario actual', 'tarifario fim mes', 'tipo', 'validacao',
       'CAV_operador', 'data', 'campanha_nome', 'estado_atual', 'dsc_operacao',
       'dif_datas_wise', 'tipo migracao', 'analise daf'],
      dtype='object')


array(['Fora Modelo Comissional', 'Retenção', 'ONE', 'TP', 'INB&WEB'],
      dtype=object)

In [ ]:
#criar zip file com report de migrações
zip_filename = 'dataframe.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
  zipf.write(csv_filename)



In [ ]:
#exportar zip file
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>